In [1]:
from validate_coarse_to_fine import * 
from dataloader.BSD500 import BSD500
import torch
from R_network_mm import RNet
import json
from matplotlib import pyplot as plt
from torchmetrics.functional import peak_signal_noise_ratio as psnr
from torchmetrics.functional import structural_similarity_index_measure as ssim

In [2]:
from skimage.metrics import peak_signal_noise_ratio as compare_psnr

def compute_PSNR(img, imclean, data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    PSNR = compare_psnr(Iclean[0,:,:,:], Img[0,:,:,:], data_range=data_range)
    return PSNR
    

In [3]:
test_dataset = BSD500('data/test.h5')

In [4]:
sigma = 5

val_data = pd.read_csv(f"{'denoise_val_results'}/validation_scores_{'sigma_5_mm_valid'}.csv").reset_index(drop=True)
p1 = val_data.loc[val_data["psnr"].idxmax()]["p1"]

exp = "/home/pourya/mm_final/exps/sigma5/64_2_7_1e-3_mm"
path_ckp = exp + "/checkpoints/checkpoint_best_epoch.pth"
path_config = exp + "/config.json"
device = 'cuda:0'
config = json.load(open(path_config))
ckp = torch.load(path_ckp, map_location={'cuda:0':device,'cuda:1':device,'cuda:2':device,'cuda:3':device})

model = RNet(config['model_params'])
model.to(device)
model.load_state_dict(ckp['state_dict'])
model.eval()

model.W2.W1.weight.data = model.sumtoone(model.W2.W1.weight)
for i, _ in enumerate(model.W2.W1s): 
    model.W2.W1s[i].weight.data = model.sumtoone(model.W2.W1s[i].weight)


model.W1.W1.weight.data = model.zeromean(model.W1.W1.weight)
for i, _ in enumerate(model.W1.W1s): 
    model.W1.W1s[i].weight.data = model.zeromean(model.W1.W1s[i].weight)


model.eigenimage = model.eigenimage.to(device)
print(model.lmbda, p1)
lip, model.eigenimage = model.W1.cal_lip(model.eigenimage, 1000)
model.alpha = 1 / lip
model.lmbda = torch.nn.Parameter(torch.ones(1, 1).to(device) * p1)


Parameter containing:
tensor([[0.0006]], device='cuda:0', requires_grad=True) 0.000534174614761


In [5]:
def mmr_denoise(y):
    s = 1e-3
    n = 5
    e = 1e-5
    q = (e/s) ** (1/n)
    tols = [s * q**(i) for i in range(n+1)]
    n_out = 10
    for _ in range(n_out-(n+1)):
        tols.append(e)
    with torch.no_grad(): 
        n_in = 100  
        c_k = model.prox_denoise_no_mask(y, y, n_in, tols[0], check_tol=True)
        for it in range(n_out - 1):   
            model.cal_mask(c_k)
            c_k = model.prox_denoise_with_mask(y, c_k, n_in, tols[it+1], check_tol=True)

    return c_k

In [6]:
i = 0
psnrs_all = list()

with torch.no_grad():
    for runs in range(5):
        psnrs = list()
        for img in test_dataset:
            i = i + 1
            if True:
                gt = img.to(device)[None, :, :, :]
                noisy_image = (img.to(device) + (5/255.0)*torch.randn(img.shape, device=device))[None, :, :, :]
                denoised =mmr_denoise(noisy_image)
                psnr = np.round(compute_PSNR(denoised, gt, 1), 2)
                psnrs.append(psnr)
                print(i, psnr)
                
        psnrs_all.append(psnrs)

1 38.23
2 36.97
3 39.23
4 38.17
5 38.26
6 36.35
7 37.81
8 37.59
9 38.82
10 37.78
11 37.1
12 35.95
13 38.06
14 36.14
15 37.29
16 38.69
17 36.86
18 36.97
19 38.38
20 37.85
21 40.73
22 35.35
23 35.52
24 35.41
25 37.56
26 37.88
27 36.96
28 40.6
29 36.54
30 37.67
31 36.92
32 38.99
33 37.96
34 34.72
35 37.36
36 37.45
37 37.43
38 36.95
39 40.33
40 35.73
41 37.56
42 40.38
43 38.25
44 38.9
45 36.1
46 38.2
47 37.56
48 36.05
49 35.18
50 44.67
51 36.97
52 39.2
53 41.08
54 38.88
55 37.56
56 36.48
57 36.15
58 36.41
59 34.88
60 36.09
61 37.01
62 36.82
63 38.14
64 35.0
65 38.14
66 40.89
67 40.06
68 38.86
69 38.23
70 36.94
71 39.23
72 38.14
73 38.29
74 36.33
75 37.81
76 37.57
77 38.81
78 37.8
79 37.05
80 35.97
81 38.06
82 36.09
83 37.25
84 38.71
85 36.89
86 36.94
87 38.38
88 37.81
89 40.7
90 35.29
91 35.48
92 35.43
93 37.62
94 37.85
95 36.96
96 40.63
97 36.54
98 37.72
99 36.87
100 38.93
101 37.91
102 34.75
103 37.37
104 37.44
105 37.43
106 36.92
107 40.38
108 35.7
109 37.5
110 40.37
111 38.22
112 38.93

In [7]:
psnr_mat = np.zeros((5, 68))

for i in range(5):
    psnr_mat[i, :] = np.array(psnrs_all[i])

std_vec = np.std(psnr_mat, axis=0)
avg_vec = np.mean(psnr_mat, axis=0)

print('std mean:', np.mean(std_vec))
print('mean mean:', np.mean(avg_vec))

std mean: 0.01826999954337279
mean mean: 37.67247058823529
